In [4]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta, datetime
import time
import plotly.express as px

# Lấy data từ API của WeatherBit cho chất lượng không khí
thông tin được lấy ở đây là thông số không khí từ ngày 1/1/2023  đến ngày 1/12/2025 tại Cần Thơ

In [5]:
API_KEY = "7db8deefb8794f2eae88d63d31f90eee"
BASE_URL_AIRQUALITY = "https://api.weatherbit.io/v2.0/history/airquality"
LAT = 10.047264
LON = 105.768115
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2025, 12, 1)
FORMAT_DATE = "%Y-%m-%d"
all_data = []


In [7]:
current = START_DATE
while current < END_DATE:
    block_end = current + timedelta(days=29)
    if block_end > END_DATE:
        block_end = END_DATE

    params = {
        "lat": LAT, 
        "lon": LON,
        "start_date": current.strftime(FORMAT_DATE),
        "end_date": block_end.strftime(FORMAT_DATE),
        "key": API_KEY
    }
    print(f"Fetching data from {params['start_date']} to {params['end_date']}")
    response = requests.get(BASE_URL_AIRQUALITY, params=params)

    if response.status_code == 200:
        json_data = response.json()
        if json_data.get("data"):
            all_data.extend(json_data["data"])
            print(f"Retrieved {len(json_data['data'])} records.")
    else:
        print(f"No data found for the period {params['start_date']} to {params['end_date']}")
        break
    
    current = block_end + timedelta(days=1)
    time.sleep(7)

Fetching data from 2023-01-01 to 2023-01-30
No data found for the period 2023-01-01 to 2023-01-30


In [8]:
df = pd.DataFrame(all_data)
df.drop(columns=["datetime"], inplace=True) 
# xử lý time series
df["timestamp_local"] = pd.to_datetime(df["timestamp_local"])
df = df.sort_values("timestamp_local").drop_duplicates()
df.set_index("timestamp_local", inplace=True)

KeyError: "['datetime'] not found in axis"

In [29]:
df.to_csv("cantho_air_quality_2023_2025.csv", index=True)

In [20]:
df.head()

,aqi,co,no2,o3,pm10,pm25,so2,timestamp_utc,ts
timestamp_local,,,,,,,,,
2023-01-01 00:00:00,152,432.8,57.3,22.3,83.3,56.67,37.7,2022-12-31T17:00:00,1672506000
2023-01-01 01:00:00,150,402.1,54.0,20.0,79.0,54.00,32.0,2022-12-31T18:00:00,1672509600
2023-01-01 02:00:00,134,361.2,45.3,24.3,70.3,48.00,31.7,2022-12-31T19:00:00,1672513200
2023-01-01 03:00:00,118,320.4,36.7,28.7,61.7,42.00,31.3,2022-12-31T20:00:00,1672516800
2023-01-01 04:00:00,102,279.5,28.0,33.0,53.0,36.00,31.0,2022-12-31T21:00:00,1672520400


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20661 entries, 2023-01-01 00:00:00 to 2025-11-30 00:00:00
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   aqi            20661 non-null  int64  
 1   co             20661 non-null  float64
 2   no2            20661 non-null  float64
 3   o3             20661 non-null  float64
 4   pm10           20661 non-null  float64
 5   pm25           20661 non-null  float64
 6   so2            20661 non-null  float64
 7   timestamp_utc  20661 non-null  object 
 8   ts             20661 non-null  int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 1.6+ MB


# Lấy dữ liệu từ thời tiết

In [2]:
API_KEY = "82a8878ff99445ecb88532871a447350"
BASE_URL_HOURLY = "https://api.weatherbit.io/v2.0/history/hourly"
##10.839318, 106.647339
LAT = 10.839318
LON = 106.647339

In [5]:

from dateutil.relativedelta import relativedelta
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2025, 12, 1)

all_data_weather_hourly = []
current = START_DATE


while current < END_DATE:
    # 👉 cộng 1 tháng
    block_end = current + relativedelta(months=1)

    if block_end > END_DATE:
        block_end = END_DATE

    params = {
        "lat": LAT,
        "lon": LON,
        "start_date": current.strftime("%Y-%m-%d"),
        "end_date": block_end.strftime("%Y-%m-%d"),
        "key": API_KEY
    }

    print(f"Fetching data from {params['start_date']} to {params['end_date']}")
    response = requests.get(BASE_URL_HOURLY, params=params)

    if response.status_code == 200:
        json_data = response.json()
        if json_data.get("data"):
            all_data_weather_hourly.extend(json_data["data"])
            print(f"Fetched {len(json_data['data'])} records")
        else:
            print("No data found for this period")
    else:
        print(f"Failed: {response.status_code} - {response.text}")
        break

    # 👉 sang tháng tiếp theo (KHÔNG cộng thêm lần nữa)
    current = block_end
    time.sleep(7)


Fetching data from 2023-01-01 to 2023-02-01
Fetched 744 records
Fetching data from 2023-02-01 to 2023-03-01
Fetched 672 records
Fetching data from 2023-03-01 to 2023-04-01
Fetched 744 records
Fetching data from 2023-04-01 to 2023-05-01
Fetched 720 records
Fetching data from 2023-05-01 to 2023-06-01
Fetched 744 records
Fetching data from 2023-06-01 to 2023-07-01
Fetched 720 records
Fetching data from 2023-07-01 to 2023-08-01
Fetched 744 records
Fetching data from 2023-08-01 to 2023-09-01
Fetched 744 records
Fetching data from 2023-09-01 to 2023-10-01
Fetched 720 records
Fetching data from 2023-10-01 to 2023-11-01
Fetched 744 records
Fetching data from 2023-11-01 to 2023-12-01
Fetched 720 records
Fetching data from 2023-12-01 to 2024-01-01
Fetched 744 records
Fetching data from 2024-01-01 to 2024-02-01
Fetched 744 records
Fetching data from 2024-02-01 to 2024-03-01
Fetched 696 records
Fetching data from 2024-03-01 to 2024-04-01
Failed: 429 - 


In [7]:
df_weather_hourly = pd.DataFrame(all_data_weather_hourly)
df_weather_hourly

,app_temp,azimuth,clouds,datetime,dewpt,dhi,dni,elev_angle,ghi,h_angle,...,temp,timestamp_local,timestamp_utc,ts,uv,vis,weather,wind_dir,wind_gust_spd,wind_spd
0,23.5,116.0,87,2023-01-01:00,20.0,55,492,10.0,132,None,...,23.0,2023-01-01T07:00:00,2023-01-01T00:00:00,1672531200,0.7,8,"{'code': 804, 'description': 'Overcast clouds'...",338,3.6,1.50
1,23.5,120.9,87,2023-01-01:01,20.0,86,735,23.0,366,None,...,23.0,2023-01-01T08:00:00,2023-01-01T01:00:00,1672534800,0.9,8,"{'code': 804, 'description': 'Overcast clouds'...",10,4.8,2.10
2,25.5,128.4,87,2023-01-01:02,19.8,104,849,35.1,586,None,...,25.0,2023-01-01T09:00:00,2023-01-01T02:00:00,1672538400,1.4,10,"{'code': 804, 'description': 'Overcast clouds'...",360,6.0,3.60
3,28.5,140.1,87,2023-01-01:03,19.9,116,910,45.7,760,None,...,27.0,2023-01-01T10:00:00,2023-01-01T03:00:00,1672542000,2.4,10,"{'code': 804, 'description': 'Overcast clouds'...",20,6.4,4.09
4,30.7,157.9,50,2023-01-01:04,19.9,122,941,53.4,870,None,...,29.0,2023-01-01T11:00:00,2023-01-01T04:00:00,1672545600,3.1,10,"{'code': 803, 'description': 'Broken clouds', ...",10,6.0,3.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,28.2,85.9,25,2024-02-29:19,18.8,0,0,-61.6,0,None,...,27.0,2024-03-01T02:00:00,2024-02-29T19:00:00,1709233200,0.0,10,"{'code': 802, 'description': 'Scattered clouds...",140,5.6,4.09
10196,28.2,89.5,25,2024-02-29:20,18.8,0,0,-46.9,0,None,...,27.0,2024-03-01T03:00:00,2024-02-29T20:00:00,1709236800,0.0,10,"{'code': 802, 'description': 'Scattered clouds...",130,5.6,3.10
10197,28.2,92.2,50,2024-02-29:21,18.8,0,0,-32.2,0,None,...,27.0,2024-03-01T04:00:00,2024-02-29T21:00:00,1709240400,0.0,10,"{'code': 803, 'description': 'Broken clouds', ...",130,5.2,3.60
10198,28.5,94.6,50,2024-02-29:22,19.9,0,0,-17.5,0,None,...,27.0,2024-03-01T05:00:00,2024-02-29T22:00:00,1709244000,0.0,10,"{'code': 803, 'description': 'Broken clouds', ...",120,4.8,3.60


In [ ]:
df_weather_hourly = pd.DataFrame(all_data_weather_hourly)
df_weather_hourly.to_csv("weather_hourly_HCM.csv", index=False)
df_weather_hourly.describe()

,app_temp,azimuth,clouds,dewpt,dhi,dni,elev_angle,ghi,precip,pres,...,slp,snow,solar_rad,temp,ts,uv,vis,wind_dir,wind_gust_spd,wind_spd
count,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,...,25560.000000,25560.0,25560.000000,25560.000000,2.556000e+04,25560.000000,25553.000000,25560.000000,25560.000000,25560.000000
mean,28.143912,179.760790,68.709351,20.820286,47.678717,379.940219,0.534441,292.053013,0.320573,1009.843897,...,1010.739163,0.0,157.541980,25.170023,1.718537e+09,1.319382,10.404414,177.102895,4.503270,1.682111
std,8.479204,98.672888,33.522666,5.967817,51.910716,407.303831,44.516869,369.852342,1.154177,7.411808,...,7.406799,0.0,237.588519,5.481672,2.656325e+07,1.918326,2.801154,99.166248,2.132712,0.848387
min,6.600000,0.000000,0.000000,-6.900000,0.000000,0.000000,-88.600000,0.000000,0.000000,983.000000,...,984.000000,0.0,0.000000,7.000000,1.672531e+09,0.000000,0.000000,0.000000,0.300000,0.000000
25%,21.600000,88.000000,50.000000,17.400000,0.000000,0.000000,-37.800000,0.000000,0.000000,1004.000000,...,1005.000000,0.0,0.000000,21.300000,1.695534e+09,0.000000,10.000000,115.000000,2.800000,1.000000
50%,26.900000,180.000000,79.000000,23.100000,15.000000,85.000000,1.400000,9.500000,0.000000,1009.000000,...,1010.000000,0.0,18.000000,26.000000,1.718537e+09,0.600000,10.000000,144.000000,4.400000,1.660000
75%,34.900000,271.800000,100.000000,25.400000,107.000000,858.000000,38.500000,635.000000,0.000000,1016.000000,...,1016.000000,0.0,208.000000,29.000000,1.741540e+09,2.100000,10.000000,255.000000,5.900000,2.000000
max,52.700000,360.000000,100.000000,29.600000,128.000000,956.000000,89.300000,1062.000000,41.000000,1033.000000,...,1034.000000,0.0,1008.000000,40.000000,1.764544e+09,12.100000,16.000000,360.000000,25.600000,12.400000
